In [23]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import category_encoders as ce
import xgboost as xgb
from sklearn.model_selection import train_test_split
from typing import Tuple

import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Import vehicle data

In [17]:
vehicle_filepath = "C:\\Users\\rcaldwell\\Documents\\modelling-club-team-1\\data\\Vehicles\\"
vehicle_files = os.listdir("C:\\Users\\rcaldwell\\Documents\\modelling-club-team-1\\data\\Vehicles")

In [18]:
# Read in vehicle data
vehicle_data = pd.read_csv(vehicle_filepath+vehicle_files[0])

for x in vehicle_files[1:len(vehicle_files)]:
    vehicle_data = vehicle_data.append(
        pd.read_csv(vehicle_filepath+x)
    )

In [24]:
vehicle_data.head(10)

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,Hit_Object_off_Carriageway,1st_Point_of_Impact,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
0,200501BS00001,1,9,0,18,0,0,0,0,0,0,1,1,15,2,74,-1,-1,-1,7,1
1,200501BS00002,1,11,0,4,0,3,0,0,0,0,4,1,1,1,42,8268,2,3,-1,-1
2,200501BS00003,1,11,0,17,0,0,0,4,0,0,4,1,1,1,35,8300,2,5,2,1
3,200501BS00003,2,9,0,2,0,0,0,0,0,0,3,1,15,1,62,1762,1,6,1,1
4,200501BS00004,1,9,0,18,0,0,0,0,0,0,1,1,15,2,49,1769,1,4,2,1
5,200501BS00005,1,3,0,18,0,0,1,10,0,0,1,1,15,1,49,85,1,10,-1,-1
6,200501BS00006,1,9,0,5,0,0,0,0,0,0,0,1,15,1,51,2976,1,1,4,1
7,200501BS00006,2,3,0,18,0,0,0,0,0,0,0,1,15,2,30,124,1,2,1,1
8,200501BS00007,1,3,0,18,0,1,0,4,0,0,1,1,15,1,31,-1,-1,-1,-1,-1
9,200501BS00007,2,9,0,2,0,1,0,0,0,0,2,1,15,1,41,4266,1,4,6,1


In [21]:
vehicle_data = vehicle_data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

### Feature encoding

In [57]:
def func_name(f):
    return(f.__name__)




def encode_features(data_frame,*list_of_encodings):
    function_list = {}
    #function_list['c_d_prodsum'] = (x['c'] * x['d']).sum()
    
    for x in list_of_encodings:
        x(data_frame,function_list)
    
    
    return pd.Series(function_list, index=func_name(*list_of_encodings))

In [58]:
list_of_encodings = []  

#### Sex of driver

In [65]:
def sex_of_drivers(data_frame,function_list):
    # Count of male drivers
    function_list['count_male_drivers'] = data_frame['Sex_of_Driver'].apply(lambda x: x[x['Sex_of_Driver'] == 1]['Sex_of_Driver'].count())
    
    # Count of female drivers
    #function_list['count_female_drivers'] = data_frame['Sex_of_Driver'].apply(lambda x: x[x['Sex_of_Driver'] == 0].count())
  

list_of_encodings.append(sex_of_drivers)

#### Apply all aggregations

In [66]:
agg_vehicle = vehicle_data.head().groupby('Accident_Index').apply(encode_features, *list_of_encodings)

TypeError: 'int' object is not subscriptable

In [56]:
# Test function - works
vehicle_data.head(10).groupby('Accident_Index').apply(lambda x: x[x['Sex_of_Driver'] == 1]['Sex_of_Driver'].count())

Accident_Index
200501BS00001    0
200501BS00002    1
200501BS00003    2
200501BS00004    0
200501BS00005    1
200501BS00006    1
200501BS00007    2
dtype: int64